In [1]:
import git
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_selection import mutual_info_regression
from sklearn.inspection import permutation_importance

from sklearn_genetic import GAFeatureSelectionCV
# from sklearn_genetic.space import Categorical, Integer, Continuous

from math import ceil

**Preprocessing**

In [2]:
# Testing whether using data_consol.csv helps anything. If so, probably indicates an error in reading in or joining the separate CSVs before
repo = git.Repo('.', search_parent_directories = True)
root = repo.working_tree_dir

data_consol = pd.read_csv(root + '//data/data_consol.csv')

In [3]:
X = data_consol.filter(regex="^[0-9]+$")
bact = data_consol['pcr_bact_log']

# Note: do NOT scale X and y before splitting, since that is a data leak. Instead, use the pipeline to scale both Xs, and separately scale the y for custom scoring like RMSE.
X_train, X_test, bact_train_unscaled, bact_test_unscaled = train_test_split(X.to_numpy(), bact.to_numpy(), train_size=0.8, random_state=0)

# Reshaping necessary for the y scaling step
bact_train_unscaled = bact_train_unscaled.reshape(-1,1)
bact_test_unscaled = bact_test_unscaled.reshape(-1,1)

bact_scaler = StandardScaler()
bact_train = bact_scaler.fit_transform(bact_train_unscaled).reshape(-1,1)
bact_test = bact_scaler.transform(bact_test_unscaled).reshape(-1,1)

# 5-fold CV; random state 0
cv_5_0 = KFold(n_splits=5, shuffle=True, random_state=0)

# Used for waveband selection
wvs = np.arange(350,2501)

rng = np.random.default_rng(0)

In [20]:
# Since this is only with respect to X_train, not any of the target variables, this only has to be computed once. (It's relatively cheap to compute, but this also has the benefit of preserving the random choices.)
def cluster(X_train):
    """ Uses agglomerative clustering with a distance threshold of 0.999 on the normalized feature correlation coefficient matrix. Then, it randomly selects one waveband from each cluster.
    This should be used as a preprocessing step when doing permutation importance. (Clustering method) """
    corr = np.corrcoef(X.T) # X needs to be transposed because of corrcoef's implementation
    agg = AgglomerativeClustering(n_clusters=None, distance_threshold=0.999) # The distance threshold is somewhat arbitrary, but it's based on EDA and domain knowledge, and the results seem reasonable.
    clusters = agg.fit_predict(corr)
    # Now select a single "representative" waveband from each cluster
    cluster_choices = []
    for i in range(np.max(clusters)):
        wv_in_cluster = wvs[clusters==i]
        cluster_choices.append(rng.choice(wv_in_cluster))
    cluster_choices = np.sort(np.array(cluster_choices))
    return cluster_choices

In [21]:
cluster_choices = cluster(X_train)

**The major pipeline components**

In [4]:
elastic_net = ElasticNet(fit_intercept=False, warm_start=True, random_state=0, selection='random', max_iter=4000)

# ga_selector = GAFeatureSelectionCV(
#     estimator=elastic_net,
#     cv=cv_5_0,  # Cross-validation folds
#     scoring="neg_root_mean_squared_error",  # Fitness function (maximize accuracy)
#     population_size=20,  # Number of individuals in the population
#     generations=50,  # Number of generations
#     n_jobs=-1,  # Use all available CPU cores
#     verbose=True,  # Print progress
#     max_features = 32
# )

# Used for embedded feature importance (via coeffs) and wrapper feature importance (via perm importance)
pipe_elastic_net = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("elastic_net", elastic_net)
    ],
    memory = root+'\\cache',
    verbose=True
)

# pipe_ga = Pipeline(
#     [
#         ("scaler", StandardScaler()),
#         ("features", ga_selector)
#     ], 
#     memory = root+'\\cache',
#     verbose=True
# )

# Hyperparameters for elastic net tuning. When code is finalized, expand for more thorough search using more computational resources.
REGULARIZATION = np.logspace(-5, 0, 8)
MIXTURE = np.linspace(0.001, 1, 8)
PARAM_GRID = [
    {
        "elastic_net__alpha": REGULARIZATION,
        "elastic_net__l1_ratio": MIXTURE
    }
]

**Train the model(s)**

In [5]:
def mi(X_train, y_train, n_features=64):
    """ Uses mutual information to calculate the n_features most related features in X_train to y_train. (Filter method) """
    y_train = y_train.ravel()
    mi = mutual_info_regression(X_train, y_train)
    top_n_idx = np.argpartition(mi, -n_features)[-n_features:]
    return wvs[top_n_idx]

In [6]:
def coeffs(X_train, y_train, n_features=64):
    """ Builds and fits an elastic net model using all features. Returns the n_features features with the highest absolute-valued coefficients. (Embedded method) """
    grid = GridSearchCV(estimator=pipe_elastic_net, param_grid=PARAM_GRID, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=cv_5_0, error_score='raise')
    grid.fit(X_train, y_train)
    coeffs = grid.best_estimator_['elastic_net'].coef_
    abs_coeffs = np.abs(coeffs)
    top_n_idx = np.argpartition(abs_coeffs, -n_features)[-n_features:]
    return wvs[top_n_idx]

In [29]:
def ga(X_train, y_train, n_features=64):
    """ Uses a genetic algorithm to find the wavebands that gives the lowest RMSE on an elastic net model. 
    The subset will be at most n_features large, but it may be less than n_features large. (GA method) """
    y_train = y_train.ravel()
    ga_selector = GAFeatureSelectionCV(
        estimator=elastic_net,
        cv=cv_5_0,  # Cross-validation folds
        scoring="neg_root_mean_squared_error",  # Fitness function (maximize accuracy)
        population_size=ceil(n_features*1.5),  # Number of individuals in the population
        generations=50,  # Number of generations
        n_jobs=-1,  # Use all available CPU cores
        verbose=True,  # Print progress
        max_features=n_features
    )
    pipe_ga = Pipeline(
        [
            ("scaler", StandardScaler()),
            ("ga", ga_selector)
        ], 
        memory = root+'\\cache',
        verbose=True
    )
    pipe_ga.fit(X_train, y_train)
    feats = pipe_ga['ga'].best_features_
    return wvs[feats]

In [26]:
def perm_imp(X_train, y_train, n_features=64):
    """ Calculates permutation importance on a dataset. cluster_choices should be the result of calling cluster(), which should be done once at the start of execution. 
    This is done outside this function to preserve the random selection. Returns the set of n_features wavebands with the highest permutation importance on the training set. """
    # Use only the features selected by clustering
    cluster_idx = cluster_choices - 350
    X_train = X_train[:,cluster_idx]
    # Build and train another elastic net model, but only on the features left after clustering, to use for permutation importance.
    pipe = Pipeline(
        [
            ("scaler", StandardScaler()),
            ("elastic_net", elastic_net)
        ], 
        memory = root+'\\cache',
        verbose=True
    )    
    grid = GridSearchCV(estimator=pipe, param_grid=PARAM_GRID, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=cv_5_0, error_score='raise')
    grid.fit(X_train, bact_train)
    perm_imp = permutation_importance(grid, X_train, bact_train, scoring='neg_root_mean_squared_error', n_repeats=10, n_jobs=-1, random_state=0)
    pi_top_64_idx = np.argpartition(perm_imp.importances_mean, -64)[-64:]
    return cluster_choices[pi_top_64_idx]

**Investigate results**

In [9]:
# print('Training RMSE:', round(abs(elastic_net_grid.score(X_train, bact_train)), 3))
# print('Testing RMSE:', round(abs(elastic_net_grid.score(X_test, bact_test)), 3))

# # Inverse-transforming the preds to get back to original scale.
# # Used for comparison with R results
# preds_unscaled = bact_scaler.inverse_transform(elastic_net_grid.predict(X_test).reshape(-1,1))
# print('Testing RMSE, unscaled:', round(root_mean_squared_error(preds_unscaled, bact_test_unscaled), 3))